In [1]:
from src.dataset import ClassficationDataset as EvaluationDataset,load_from_json
from src.model import Encoder
from torch.utils.data import DataLoader
from tqdm import tqdm

/home/avishka/anaconda3/envs/contraclassi/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from sklearn.cluster import KMeans
from sklearn.metrics import accuracy_score
from scipy.optimize import linear_sum_assignment
import numpy as np

def clustering_accuracy(embeddings, true_labels, n_clusters):
    kmeans = KMeans(n_clusters=n_clusters, random_state=0).fit(embeddings)
    cluster_labels = kmeans.labels_

    cost_matrix = np.zeros((n_clusters, n_clusters), dtype=int)
    for i in range(n_clusters):
        for j in range(n_clusters):
            cost_matrix[i, j] = -np.sum((cluster_labels == i) & (true_labels == j))

    row_ind, col_ind = linear_sum_assignment(cost_matrix)

    label_mapping = {row: col for row, col in zip(row_ind, col_ind)}

    predicted_labels = np.array([label_mapping[label] for label in cluster_labels])

    accuracy = accuracy_score(true_labels, predicted_labels)
    
    return accuracy


In [7]:
data_dir = "/home/avishka/muditha/contra_classi/data/annots"
split = "pet_test"

std_ds = EvaluationDataset(data_dir, split)
std_dl = DataLoader(std_ds, batch_size=4, shuffle=False, pin_memory=True)


In [8]:
base_model = "convnext_tiny"
version = "fb_in22k"
ckpt = "/home/avishka/muditha/contra_classi/logs/contrastive_convnext_pet/run0/ckpts/convnext_tiny_encoder.pt"
enc = Encoder(base_model,version,1,ckpt)

Loaded model weights successfully!!


In [9]:
enc.eval()
embs = []
labels = []
# std_embs = {}

for _, batch in tqdm(
    enumerate(std_dl), desc="Evaluation", total=len(std_dl)
):
    emb = enc(batch[0])
    label = batch[1][0]

    for i, e in enumerate(emb):
        embs.append(e.cpu().detach().numpy().reshape(-1))
        labels.append(label)
embed = np.array(embs)
labels = np.array(labels)

Evaluation: 100%|██████████| 370/370 [00:11<00:00, 32.50it/s]


In [15]:
clustering_accuracy(embed,labels,5)

/home/avishka/anaconda3/envs/contraclassi/lib/python3.11/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


0.9743243243243244

In [18]:
clustering_accuracy(embed,labels,5)

/home/avishka/anaconda3/envs/contraclassi/lib/python3.11/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


0.8554054054054054

In [10]:
clustering_accuracy(embed,labels,37)

/home/avishka/anaconda3/envs/contraclassi/lib/python3.11/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


0.9600811907983762

In [12]:
clustering_accuracy(embed,labels,37)

/home/avishka/anaconda3/envs/contraclassi/lib/python3.11/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


0.8897158322056834

In [6]:
#ld
clustering_accuracy(embed,labels,5)

/home/avishka/anaconda3/envs/contraclassi/lib/python3.11/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


0.5666666666666667

In [10]:
clustering_accuracy(embed,labels,5)

/home/avishka/anaconda3/envs/contraclassi/lib/python3.11/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


0.6666666666666666

In [18]:
#pet
clustering_accuracy(embed,labels,37)

/home/avishka/anaconda3/envs/contraclassi/lib/python3.11/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


0.5540540540540541

In [6]:
#pet_trip
clustering_accuracy(embed,labels,37)

/home/avishka/anaconda3/envs/contraclassi/lib/python3.11/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


0.8243243243243243